In [1]:
import csv
import os
import gc
import numpy as np
import tensorflow as tf
from random import randint
from tensorflow import keras
from keras.utils import np_utils, plot_model, to_categorical
from keras.models import Model
from keras.layers import Input, GRU, Dense, RepeatVector, TimeDistributed, concatenate

In [15]:
# returns train, inference_encoder and inference_decoder models
def define_models(dim_p, n_units):
	# define training encoder
	enc_in_o = Input(shape=(None, 1))
	enc_in_q = Input(shape=(None, 1))
	enc_in_p = Input(shape=(None, dim_p))
	encoder_inputs = concatenate([enc_in_o, enc_in_q, enc_in_p])
	encoder = GRU(n_units, return_state=True)
	encoder_outputs, state_h = encoder(encoder_inputs)
	
	# define training decoder
	dec_in_o = Input(shape=(None, 1))
	dec_in_q = Input(shape=(None, 1))
	dec_in_p = Input(shape=(None, dim_p))
	decoder_inputs = concatenate([dec_in_o, dec_in_q, dec_in_p])
	decoder_gru = GRU(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=state_h)
	dec_dense_o = Dense(1, activation='relu', name='tr_out_o')
	dec_dense_q = Dense(1, activation='sigmoid', name='tr_out_q')
	dec_dense_p = Dense(dim_p, activation='softmax', name='tr_out_p')
	#out_o = Dense(1, activation='relu', name='tr_out_o')(decoder_outputs)#act relu
	#out_q = Dense(1, activation='sigmoid', name='tr_out_q')(decoder_outputs)
	#out_p = Dense(dim_p, activation='softmax', name='tr_out_p')(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)
	
	model = Model([enc_in_o, enc_in_q, enc_in_p, dec_in_o, dec_in_q, dec_in_p], [out_o, out_q, out_p])
	
	# define inference encoder
	encoder_model = Model([enc_in_o, enc_in_q, enc_in_p], state_h)
	
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_outputs, state_h = decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)
	#out_o = TimeDistributed(Dense(1, activation='relu'))(decoder_outputs)#act relu
	#out_q = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)
	#out_p = TimeDistributed(Dense(dim_p, activation='softmax'))(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)
	
	decoder_model = Model([dec_in_o, dec_in_q, dec_in_p] + [decoder_state_input_h], [out_o, out_q, out_p] + [state_h])
	# return all models
	return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, src_o, src_q, src_p, n_steps, cardinality):
	# encode
	state = infenc.predict([src_o, src_q, src_p])
	# start of sequence input
	target_o = np.array([-1]).reshape(1, 1, 1)
	target_q = np.array([-1]).reshape(1, 1, 1)
	target_p = 0
	target_p = to_categorical(target_p, cardinality).reshape((1,1,-1))
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		#print(target_o.shape)
		#print(target_q.shape)
		#print(target_p.shape)
		#print(state[0].shape)
		o, q, p, h, = infdec.predict([target_o, target_q, target_p] + [state])
		#print(a)
		# store prediction
		output.append(o[0,0,:])
		output.append(q[0,0,:])
		output.append(p[0,0,:])
		# update state
		state = h
		# update target sequence
		target_o = o
		target_q = q
		target_p = p
	return np.array(output)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

In [3]:
#load data
stream_list = []

for directory, subdirectories, files in os.walk('data'):
    for file in files:
        with open('data/' + file, 'r') as f: 
            reader = csv.reader(f)
            sub_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]
            stream_list = stream_list + sub_list

In [4]:
#create seperate data structures for each variable (offset, quarterlength, pitch)
#normalise offset and quarterlength
offs = []
qlngth = []
ptch = []

offsb = max(element[0] for element in stream_list)
qlngthb = max(element[1] for element in stream_list)
#ptchb = 127.0

for row in stream_list:
    offs.append(row[0]/offsb)
    qlngth.append(row[1]/qlngthb)
    ptch.append(row[2])

In [ ]:
del stream_list
gc.collect()

128

In [5]:
#divide the sets in sequences of specific length 
dtlngth=len(offs)
seq_length = 4#100 groups of 3

dataX1_o = []
dataX1_q = []
dataX1_p = []
dataX2_o = []
dataX2_q = []
dataX2_p = []

for i in range(0, dtlngth - seq_length + 1, 1):
	seq_in_o = offs[i:i + seq_length]
	seq_in_q = qlngth[i:i + seq_length]
	seq_in_p = ptch[i:i + seq_length]

	# create padded input target sequence
	target_in_o = [-1] + seq_in_o[:-1]
	target_in_q = [-1] + seq_in_q[:-1]
	target_in_p = [0] + seq_in_p[:-1]
	
	src_encoded = to_categorical(seq_in_p, num_classes=127+1)
	tar2_encoded = to_categorical(target_in_p, num_classes=127+1)
	
	dataX1_o.append([offs for offs in seq_in_o])
	dataX1_q.append([qlngth for qlngth in seq_in_q])
	dataX1_p.append(src_encoded)
	dataX2_o.append([offs for offs in target_in_o])
	dataX2_q.append([qlngth for qlngth in target_in_q])
	dataX2_p.append(tar2_encoded)

n_patterns = len(dataX1_p)
print ("Total Patterns: ", n_patterns)

Total Patterns:  32385


In [6]:
#reshape inputs to be [samples, time steps, features], one hot encode pitch output
dataX1_o = np.reshape(dataX1_o, (n_patterns, seq_length, 1))
dataX1_q = np.reshape(dataX1_q, (n_patterns, seq_length, 1))

dataX2_o = np.reshape(dataX2_o, (n_patterns, seq_length, 1))
dataX2_q = np.reshape(dataX2_q, (n_patterns, seq_length, 1))

In [ ]:
del offs
del qlngth
del ptch
gc.collect()

44

In [7]:
#divide data in train and validation sets
dataX1_o_tr = dataX1_o[:-1000]
dataX1_o_v = dataX1_o[-1000:]
dataX2_o_tr = dataX2_o[:-1000]
dataX2_o_v = dataX2_o[-1000:]

dataX1_q_tr = dataX1_q[:-1000]
dataX1_q_v = dataX1_q[-1000:]
dataX2_q_tr = dataX2_q[:-1000]
dataX2_q_v = dataX2_q[-1000:]

dataX1_p_tr = dataX1_p[:-1000]
dataX1_p_v = dataX1_p[-1000:]
dataX2_p_tr = dataX2_p[:-1000]
dataX2_p_v = dataX2_p[-1000:]

In [16]:
# configure problem
n_features = np.array(dataX2_p).shape[2]
n_steps_in = 4
n_steps_out = 4
# define model
train, infenc, infdec = define_models(n_features, 256)
train.compile(optimizer='adam', loss={'tr_out_o': 'mse', 'tr_out_q': 'mse', 'tr_out_p': 'categorical_crossentropy'},
 metrics={'tr_out_o': 'mean_squared_error', 'tr_out_q': 'mean_squared_error', 'tr_out_p': 'accuracy'})

In [17]:
# train model
train.fit([dataX1_o_tr, dataX1_q_tr, np.array(dataX1_p_tr), dataX2_o_tr, dataX2_q_tr, np.array(dataX2_p_tr)], [dataX1_o_tr, dataX1_q_tr, np.array(dataX1_p_tr)], epochs=10)

Epoch 1/10
981/981 [==============================] - 55s 56ms/step - loss: 1.1937 - tr_out_o_loss: 0.0072 - tr_out_q_loss: 0.0038 - tr_out_p_loss: 1.1827 - tr_out_o_mean_squared_error: 0.0072 - tr_out_q_mean_squared_error: 0.0038 - tr_out_p_accuracy: 0.6799
Epoch 2/10
981/981 [==============================] - 56s 57ms/step - loss: 0.0778 - tr_out_o_loss: 0.0032 - tr_out_q_loss: 0.0014 - tr_out_p_loss: 0.0732 - tr_out_o_mean_squared_error: 0.0032 - tr_out_q_mean_squared_error: 0.0014 - tr_out_p_accuracy: 0.9868
Epoch 3/10
981/981 [==============================] - 56s 57ms/step - loss: 0.0186 - tr_out_o_loss: 0.0014 - tr_out_q_loss: 0.0013 - tr_out_p_loss: 0.0159 - tr_out_o_mean_squared_error: 0.0014 - tr_out_q_mean_squared_error: 0.0013 - tr_out_p_accuracy: 0.9984
Epoch 4/10
981/981 [==============================] - 55s 56ms/step - loss: 0.0107 - tr_out_o_loss: 8.1902e-04 - tr_out_q_loss: 0.0012 - tr_out_p_loss: 0.0087 - tr_out_o_mean_squared_error: 8.1902e-04 - tr_out_q_mean_square

In [ ]:
infdec.summary()

In [18]:
# spot check some examples
for _ in range(10):
    i = randint(1, 1000)
    X1_o = np.reshape(dataX1_o_v[i], (1, seq_length, 1))
    X1_q = np.reshape(dataX1_q_v[i], (1, seq_length, 1))
    X1_p = np.reshape(dataX1_p_v[i], (1, seq_length, n_features))
    target = predict_sequence(infenc, infdec, X1_o, X1_q, X1_p, n_steps_out, n_features)
    for j in range(seq_length):
        print('X_o=%s, y_o=%s, X_q=%s, y_q=%s, X_p=%s, y_p=%s' % (dataX1_o_v[i][j]*offsb, target[3*j]*offsb, dataX1_q_v[i][j]*qlngthb, target[3*j+1]*qlngthb, one_hot_decode([dataX1_p_v[i][j]]), one_hot_decode([target[3*j+2]])))
    print()

X_o=[452.], y_o=[449.56598], X_q=[1.5], y_q=[1.3300873], X_p=[57], y_p=[57]
X_o=[455.], y_o=[459.5011], X_q=[0.75], y_q=[0.6635941], X_p=[33], y_p=[33]
X_o=[456.], y_o=[455.61832], X_q=[1.25], y_q=[1.3308741], X_p=[69], y_p=[69]
X_o=[456.], y_o=[453.1969], X_q=[1.25], y_q=[0.96714866], X_p=[74], y_p=[74]

X_o=[481.5], y_o=[483.35266], X_q=[1.5], y_q=[0.78058094], X_p=[71], y_p=[71]
X_o=[481.5], y_o=[473.91415], X_q=[1.5], y_q=[0.5334712], X_p=[74], y_p=[74]
X_o=[481.5], y_o=[476.45978], X_q=[1.33333333], y_q=[0.59748554], X_p=[79], y_p=[79]
X_o=[481.5], y_o=[474.43454], X_q=[0.25], y_q=[0.699861], X_p=[31], y_p=[31]

X_o=[521.5], y_o=[534.6348], X_q=[1.5], y_q=[0.8565914], X_p=[74], y_p=[74]
X_o=[521.5], y_o=[531.3925], X_q=[1.5], y_q=[0.5478591], X_p=[78], y_p=[78]
X_o=[521.5], y_o=[533.4022], X_q=[0.25], y_q=[0.23902059], X_p=[35], y_p=[35]
X_o=[521.5], y_o=[550.1788], X_q=[0.5], y_q=[1.1893027], X_p=[61], y_p=[61]

X_o=[436.], y_o=[441.3416], X_q=[1.], y_q=[0.43449885], X_p=[76], y_

In [ ]:
target

array([array([0.6091928], dtype=float32),
       array([0.81667787], dtype=float32),
       array([0.02832382, 0.00506055, 0.00252124, 0.00338739, 0.00328134,
       0.0043051 , 0.00730426, 0.00505911, 0.00732537, 0.00692601,
       0.01266262, 0.00638036, 0.00602316, 0.00630103, 0.00870636,
       0.00274761, 0.00915835, 0.00475388, 0.01208431, 0.00919578,
       0.01092449, 0.00631092, 0.01472623, 0.00756396, 0.00691384,
       0.00448319, 0.00763378, 0.00508943, 0.00458313, 0.0024238 ,
       0.0024929 , 0.01170231, 0.00595025, 0.01534734, 0.00422082,
       0.00325   , 0.01816621, 0.00512767, 0.01175208, 0.01093599,
       0.00672235, 0.00469713, 0.02320295, 0.0085847 , 0.00402998,
       0.00713945, 0.00365347, 0.00451266, 0.01456772, 0.01473746,
       0.00876913, 0.00495217, 0.01188774, 0.01027179, 0.00510809,
       0.01228774, 0.00407863, 0.00992823, 0.00197576, 0.00431416,
       0.00362944, 0.00733132, 0.01178547, 0.00223031, 0.00690317,
       0.01109138, 0.00861269, 0.0154

In [ ]:
X1_o = np.reshape(dataX1_o[2], (1, 2, 1))
X1_q = np.reshape(dataX1_q[2], (1, 2, 1))
X1_p = np.reshape(dataX1_p[2], (1, 2, 86))
state = infenc.predict([X1_o, X1_q, X1_p])
state
#target_o = np.array([-1]).reshape(1, 1)
#target_q = np.array([-1]).reshape(1, 1)
#target_p = -1.0
#target_p = np_utils.to_categorical(target_p, num_classes=86)
#target_p = np.array(target_p.reshape(1, 86)
#yhat, h, c = infdec.predict([target_o, target_q, target_p] + state)